In [3]:
%matplotlib inline

%matplotlib inline
%load_ext autoreload
%autoreload 2

import ERFutils
import dask
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter

import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cftime
import dask
import xarrayutils
import cartopy.crs as ccrs
from xmip.preprocessing import combined_preprocessing
from xmip.preprocessing import replace_x_y_nominal_lat_lon
from xmip.drift_removal import replace_time
from xmip.postprocessing import concat_experiments
import xmip.drift_removal as xm_dr
import xmip as xm
import xesmf as xe
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import cf_xarray as cfxr
import scipy.signal as signal
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve_triangular

import seaborn as sns
import matplotlib as mpl
import cmocean
import cmocean.cm as cmo
from matplotlib.gridspec import GridSpec

dask.config.set(**{'array.slicing.split_large_chunks': True})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# Select experiments, models, and variables of interest
experiments = ['1pctCO2','piControl','historical','ssp126','ssp245','ssp370','ssp585']
models = ERFutils.model_set
variables = ['tas','rsdt','rsut','rlut']

# Check that all variables are imported
check_imported(experiments, models, variables, False)

Experiment: 1pctCO2
	Model: INM-CM5-0
		Missing: ['tas', 'rsdt', 'rsut', 'rlut']
	Model: ACCESS-CM2
		Missing: ['tas', 'rsdt', 'rsut', 'rlut']
	Model: MRI-ESM2-0
		Missing: ['tas', 'rsdt', 'rsut', 'rlut']
	Model: ACCESS-ESM1-5
		Missing: ['tas', 'rsdt', 'rsut', 'rlut']
	Model: INM-CM4-8
		Missing: ['tas', 'rsdt', 'rsut', 'rlut']
	Model: CanESM5
		Missing: ['tas', 'rsdt', 'rsut', 'rlut']
	Model: CAMS-CSM1-0
		Missing: ['tas', 'rsdt', 'rsut', 'rlut']
	Model: NorESM2-LM
		Missing: ['tas', 'rsdt', 'rsut', 'rlut']
	Model: MIROC6
		Missing: ['tas', 'rsdt', 'rsut', 'rlut']
Experiment: piControl
	Model: INM-CM5-0
		Missing: ['tas', 'rsdt', 'rsut', 'rlut']
	Model: ACCESS-CM2
		Missing: ['tas', 'rsdt', 'rsut', 'rlut']
	Model: MRI-ESM2-0
		Missing: ['tas', 'rsdt', 'rsut', 'rlut']
	Model: ACCESS-ESM1-5
		Missing: ['tas', 'rsdt', 'rsut', 'rlut']
	Model: INM-CM4-8
		Missing: ['tas', 'rsdt', 'rsut', 'rlut']
	Model: CanESM5
		Missing: ['tas', 'rsdt', 'rsut', 'rlut']
	Model: CAMS-CSM1-0
		Missing: ['ta

In [1]:
def check_imported(experiments,models,variables,print_path=False):
    #path_to_cmip6_data_local = '../../cmip6_data/'
    path_to_cmip6_data = ERFutils.path_to_cmip6_data_local
    
    failed = {}
    
    for exp_id in experiments:
        print(f'Experiment: {exp_id}') 
        failed[exp_id] = {}
        for m in models:
            print(f'\tModel: {m}')
            failed[exp_id][m] = []
            for var in variables:
                try:
                    if print_path:
                        print(f'{path_to_cmip6_data}{exp_id}/{var}_Amon_{m}_{exp_id}_r1i1p1f1**')
                    xr.open_mfdataset(f'{path_to_cmip6_data}{exp_id}/{var}_Amon_{m}_{exp_id}_r1i1p1f1**', use_cftime=True)
                except:
                    failed[exp_id][m].append(var)
                    
            if len(failed[exp_id][m]) == 0:
                print('\t\tAll variables present!')
            else:
                print(f'\t\tMissing: {[m_var for m_var in failed[exp_id][m]]}')
                
    return
    